In [37]:
n = 2**10

NCPUS=8

KK = CyclotomicField(2*n)
zeta = KK.gen()
Cn = VectorSpace(CC, KK.degree())

In [36]:
sigma = 3
data_sigma = 10
data_mu = 4
k = 2**5

In [32]:
def sample_es():
    assert n * k % NCPUS == 0
    jobs_per_cpu = n * k // NCPUS
    @parallel(ncpus=NCPUS)
    def sample_batch(sigma):
        set_random_seed()
        return [normalvariate(0, sigma) for _ in range(jobs_per_cpu)]
    vals = sample_batch([sigma for _ in range(NCPUS)])
    samples = sum((val[-1] for val in vals),[])
    return [KK(samples[i*n:(i+1)*n]) for i in range(k)]

def sample_complex_es():
    # First n/2 are Re(output), second n/2 are Im(output)
    # Extend to n-dim C vector by setting output[0] = conj(output[-1])
    assert n * k % NCPUS == 0
    jobs_per_cpu = n * k // NCPUS
    @parallel(ncpus=NCPUS)
    def sample_batch(sigma):
        set_random_seed()
        return [normalvariate(0, sigma) for _ in range(jobs_per_cpu)]
    vals = sample_batch([sigma * sqrt(n) for _ in range(NCPUS)])
    samples = sum((val[-1] for val in vals),[])
    # convert n Normals to conj C^n vecs
    def convert(sample_batch):
        output = [_ for _ in range(n)]
        for i in range(n//2):
            output[i] = CC(sample_batch[i], sample_batch[i+n//2])
            output[-i+1] = output[i].conjugate()
        return Cn(output)        
    return [convert(samples[i*n:(i+1)*n]) for i in range(k)]


def sample_xs():
    samples = [normalvariate(data_mu, data_sigma) for _ in range(k)]
    return [KK(samples[i]) for i in range(k)]

In [6]:
def l2_norm(x):
    return x.polynomial().norm(2)

def canl2_norm(x):
    return Cn(x.complex_embeddings()).norm(2)

def inf_norm(x):
    return x.polynomial().norm(infinity)

def caninf_norm(x):
    return Cn(x.complex_embeddings()).norm(infinity)

In [19]:
def normalize_elem(elem):
    return KK(elem.polynomial().map_coefficients(numerical_approx))

def compute_avg_e(es):
    k = len(es)
    avg_e = sum(es[i] for i in range(k))
    return normalize_elem(avg_e * (1/k))

def compute_complex_avg_e(complex_es):
    return (1/len(complex_es)) * sum(complex_e for complex_e in complex_es)

def embed_es(es):
    assert k % NCPUS == 0
    jobs_per_cpu = k // NCPUS
    @parallel(ncpus=NCPUS)
    def embed_batch(batch_es):
        return [batch_es[i].complex_embeddings() for i in range(jobs_per_cpu)]
    embedded_es = embed_batch([es[i*jobs_per_cpu:(i+1)*jobs_per_cpu] for i in range(NCPUS)])
    embedded_es = sum((batch[-1] for batch in embedded_es), [])
    return [Cn(embedded_e) for embedded_e in embedded_es]

def embed_avg_e(avg_e):
    return Cn(normalize_elem(avg_e).complex_embeddings())

def inf_bound(es, avg_e, xs):
    k = len(xs)
    assert k == len(es)
    return 4*k*sum(inf_norm(es[i] - avg_e) * (xs[i] + k*n*inf_norm(es[i])) for i in range(k))

def caninf_bound(embedded_es, embedded_avg_e, xs):
    # Embed polynomials, operate on embedded vectors
    k = len(xs)
    assert k == len(embedded_es)
    n = embedded_es[0].parent().degree()
    return 4*k*sum((embedded_es[i] - embedded_avg_e).norm(infinity) * (xs[i] + n * k * embedded_es[i].norm(infinity)) for i in range(k))

def canl2_bound(embedded_es, embedded_avg_e, xs):
    # Embed polynomials, operate on embedded vectors
    k = len(xs)
    assert k == len(embedded_es)
    return 4*k*sum((embedded_es[i] - embedded_avg_e).norm(2) * (xs[i] + k * embedded_es[i].norm(infinity)) for i in range(k))

def gauss_approx(es, xs):
    k = len(xs)
    assert k == len(es)
    n = es[0].parent().degree()
    xs_sqnorm = N(sum(xs[i]**2 for i in range(k)))
    return sqrt(4*sigma**2 * (2*k+1)*(xs_sqnorm + 2*n*k*sigma**2))

def actual_error(es, avg_e, xs):
    k = len(es)
    assert k == len(xs)
    assert k % NCPUS == 0
    num_jobs = k // NCPUS
    @parallel(ncpus=NCPUS)
    def summand_batch(batch_xs, batch_es):
        return sum(normalize_elem(2*batch_xs[i]+k*batch_es[i]*avg_e) for i in range(num_jobs))
    res = summand_batch([(xs[i], es[i]) for i in range(k)])
    return normalize_elem(sum(val[-1] for val in res))

def actual_complex_error(complex_es, avg_complex_e, xs):
    k = len(complex_es)
    assert k == len(xs)
    return sum(2 * Cn(xs[i].complex_embeddings()) + k * complex_es[i].pairwise_product(avg_complex_e) for i in range(k))


def data(es, avg_e, xs, embedded_es, embedded_avg_e):
    excess = lambda a,b: log_b(a/b,2)
    err = actual_error(es, avg_e, xs)
    inf_excess = excess(inf_bound(es, avg_e, xs), inf_norm(err))
    caninf_excess = excess(caninf_bound(embedded_es, embedded_avg_e, xs), caninf_norm(err))
    canl2_excess = excess(canl2_bound(embedded_es, embedded_avg_e, xs), canl2_norm(err))
    gauss_excess = excess(gauss_approx(es, xs), l2_norm(err))
    return [inf_excess, caninf_excess, canl2_excess, gauss_excess]

def trial():
    es = sample_es()
    avg_e = compute_avg_e(es)
    xs = sample_xs()
    embedded_es = embed_es(es)
    embedded_avg_e = embed_avg_e(avg_e)
    return data(es, avg_e, xs, embedded_es, embedded_avg_e)

def real_trial():
    es = sample_es()
    avg_e = compute_avg_e(es)
    xs = sample_xs()
    excess = lambda a,b: log_b(a/b,2)
    err = actual_error(es, avg_e, xs)
    inf_excess = excess(inf_bound(es, avg_e, xs), inf_norm(err))
    gauss_excess = excess(gauss_approx(es, xs), l2_norm(err))
    return [inf_excess, gauss_excess]
    

def complex_trial():
    embedded_es = sample_complex_es()
    embedded_avg_e = compute_complex_avg_e(embedded_es)
    xs = sample_xs()
    embedded_err = actual_complex_error(embedded_es, embedded_avg_e, xs)
    excess = lambda a,b: log_b(a/b,2)
    caninf_excess = excess(caninf_bound(embedded_es, embedded_avg_e, xs), embedded_err.norm(infinity))
    canl2_excess = excess(canl2_bound(embedded_es, embedded_avg_e, xs), embedded_err.norm(2))
    return [caninf_excess, canl2_excess]

def synthetic_trial():
    real = real_trial()
    cmplx = complex_trial()
    return [real[0], cmplx[0], cmplx[1], real[1]]

In [38]:
%prun -s cumulative res = trial(), synthetic_trial()

In [29]:
trial()

[21.6661330271510, 25.6198845593748, 16.8599471992810, 3.14928778414005]

In [35]:
res[0]

[21.3912660186269, 25.6773293604520, 16.7473228453930, 3.03796699790274]

In [ ]:
es = sample_es()

In [ ]:
e = es[0]

In [ ]:
embs = e.complex_embeddings()

In [ ]:
embs[-1]

In [ ]:
z = CC(0,1)

In [ ]:
x = sample_complex_es()[0]

In [ ]:
x + x

In [ ]:
x.pairwise_product?